In [1]:
# Import Regex
import re
import pandas as pd
import os

In [2]:
# Identify different formats of phone number (US)
def is_phone_number(phone):
    patterns = [
        r'^\d{3}-\d{3}-\d{4}$',            # 123-456-7890
        r'^\(\d{3}\)\s\d{3}-\d{4}$',       # (123) 456-7890
        r'^\d{3}\.\d{3}\.\d{4}$',          # 123.456.7890
        r'^\+?\d{1,2}-\d{3}-\d{3}-\d{4}$', # +1-123-456-7890 or 1-123-456-7890
        r'^\d\s?\(\d\s?\d\s?\d\)\s?\d\s?\d\s?\d\sdash\s?\d\s?\d\s?\d\s?\d$',          # 1 (123) 456-7890
        r'^\d{10}$',                       # 1234567890
        r'^\d{3}\s\d{3}\s\d{4}$',          # 123 456 7890
        r'^(?:\d\s?){10}$',                # 1 2 3 4 5 6 7 8 9 0
        r'^\(\d{3}\)\s?-\s?\d{3}-\d{4}$',  # (111)-222-3333 

       # Flexible separators: spaces, dashes, or dots
        r'^\+?\d{1,2}[\s.-]?\d{3}[\s.-]?\d{3}[\s.-]?\d{4}$',  # +1 123-456.7890 or 123 456-7890
        r'^\(\d{3}\)-\d{6,7}' # (123)-4567890
    ]
    # area_codes = [
    #     '205', '251', '256', '334', '659', '938',  # Alabama
    #     '907',  # Alaska
    #     '480', '520', '602', '623', '928',  # Arizona
    #     '327', '479', '501', '870',  # Arkansas
    #     '209', '213', '279', '310', '323', '341', '350', '369', '408', '415', '424', '442', '510', '530', '559', '562', '619', '626', '628', '650', '657', '661', '669', '707', '714', '747', '760', '805', '818', '820', '831', '840', '858', '909', '916', '925', '949', '951',  # California
    #     '303', '719', '720', '970', '983',  # Colorado
    #     '203', '475', '860', '959',  # Connecticut
    #     '302',  # Delaware
    #     '239', '305', '321', '324', '352', '386', '407', '448', '561', '645', '656', '689', '727', '728', '754', '772', '786', '813', '850', '863', '904', '941', '954',  # Florida
    #     '229', '404', '470', '478', '678', '706', '762', '770', '912', '943',  # Georgia
    #     '808',  # Hawaii
    #     '208', '986',  # Idaho
    #     '217', '224', '309', '312', '331', '447', '464', '618', '630', '708', '730', '773', '779', '815', '847', '861', '872',  # Illinois
    #     '219', '260', '317', '463', '574', '765', '812', '930',  # Indiana
    #     '319', '515', '563', '641', '712',  # Iowa
    #     '316', '620', '785', '913',  # Kansas
    #     '270', '364', '502', '606', '859',  # Kentucky
    #     '225', '318', '337', '504', '985',  # Louisiana
    #     '207',  # Maine
    #     '227', '240', '301', '410', '443', '667',  # Maryland
    #     '339', '351', '413', '508', '617', '774', '781', '857', '978',  # Massachusetts
    #     '231', '248', '269', '313', '517', '586', '616', '734', '810', '906', '947', '989',  # Michigan
    #     '218', '320', '507', '612', '651', '763', '952',  # Minnesota
    #     '228', '601', '662', '769',  # Mississippi
    #     '235', '314', '417', '557', '573', '636', '660', '816', '975',  # Missouri
    #     '406',  # Montana
    #     '308', '402', '531',  # Nebraska
    #     '702', '725', '775',  # Nevada
    #     '603',  # New Hampshire
    #     '201', '551', '609', '640', '732', '848', '856', '862', '908', '973',  # New Jersey
    #     '505', '575',  # New Mexico
    #     '212', '315', '329', '332', '347', '363', '516', '518', '585', '607', '624', '631', '646', '680', '716', '718', '838', '845', '914', '917', '929', '934',  # New York
    #     '252', '336', '472', '704', '743', '828', '910', '919', '980', '984',  # North Carolina
    #     '701',  # North Dakota
    #     '216', '220', '234', '283', '326', '330', '380', '419', '436', '440', '513', '567', '614', '740', '937',  # Ohio
    #     '405', '539', '572', '580', '918',  # Oklahoma
    #     '458', '503', '541', '971',  # Oregon
    #     '215', '223', '267', '272', '412', '445', '484', '570', '582', '610', '717', '724', '814', '835', '878',  # Pennsylvania
    #     '401',  # Rhode Island
    #     '803', '839', '843', '854', '864',  # South Carolina
    #     '605',  # South Dakota
    #     '423', '615', '629', '731', '865', '901', '931',  # Tennessee
    #     '210', '214', '254', '281', '325', '346', '361', '409', '430', '432', '469', '512', '682', '713', '726', '737', '806', '817', '830', '832', '903', '915', '936', '940', '945', '956', '972', '979',  # Texas
    #     '385', '435', '801',  # Utah
    #     '802',  # Vermont
    #     '276', '434', '540', '571', '686', '703', '757', '804', '826', '948',  # Virginia
    #     '206', '253', '360', '425', '509', '564',  # Washington
    #     '202', '771',  # Washington, DC
    #     '304', '681',  # West Virginia
    #     '262', '274', '353', '414', '534', '608', '715', '920',  # Wisconsin
    #     '307',  # Wyoming
    #     '684',  # American Samoa
    #     '671',  # Guam
    #     '670',  # Northern Mariana Islands
    #     '787', '939',  # Puerto Rico
    #     '340'  # Virgin Islands
    # ]

    # Include area code in the criteria
    # for pattern in patterns:
    #     if re.match(pattern, phone):
    #         # Extract the area code from the phone number
    #         match = re.search(r'\(?(\d{3})\)?[-\s.]?\d{3}[-\s.]?\d{4}', phone)
    #         if match:
    #             area_code = match.group(1)  # Extract the area code
    #             if area_code in area_codes:
    #                 return True  # The phone number is valid and has a valid area code
    #             else:
    #                 return False  # Invalid area code

    # Exclude area code in the criteria
    for pattern in patterns:
        if re.match(pattern, phone):
           return True  # The phone number is valid and has a valid area code
    
    return False

In [3]:
# Test 1
test_inputs = [
    # should evaluate to True
    '(217) 456-7890',
    '839-456-7890',
    '240.950.3182',
    '+1-217-456-7890',
    '5159820455',
    '(669)-246-2485',

    # should evaluate to False
    '(111)-222-3333', # technically invalid area code
    # '+553 575-381.4440', 
    # '002716236.1690', 
    # '0049 48 490.5598', 
    '+21.13 926 7829', # invalid
    '+41 15-968.9292', # invalid
    '1234-567-890'     # invalid
]

for text in test_inputs:
    print(f'{text}: {is_phone_number(text)}')

(217) 456-7890: True
839-456-7890: True
240.950.3182: True
+1-217-456-7890: True
5159820455: True
(669)-246-2485: True
(111)-222-3333: True
+21.13 926 7829: False
+41 15-968.9292: False
1234-567-890: False


In [6]:
# Test the algorithm using the dataset
filepath = os.path.join(os.getcwd(), 'data', 'extracted_tel_numbers_english.csv')
df = pd.read_csv(filepath)

FileNotFoundError: [Errno 2] No such file or directory: 'e:\\Desktop\\Extracurriculars\\Projects\\BTTAI\\AI Studio\\Amex2B\\AmexSensitiveData\\regex\\data\\extracted_tel_numbers_english.csv'

In [61]:
df.head()

,unmasked_text,masked_text,phone_number
0,"Mr. Franecki, we have scheduled a learning ass...","[PREFIX_1] [LASTNAME_1], we have scheduled a l...",(612).3785804
1,The palliative care team is coordinating a hom...,The palliative care team is coordinating a hom...,09.48-67 61 36
2,"We've updated your next appointment, Tavares. ...","We've updated your next appointment, [FIRSTNAM...",+19-302 725.8274
3,Lawrence County Sports department is conductin...,[COUNTY_1] Sports department is conducting a p...,+97-259 289.7050
4,"Dear Ivory, Your appointment with Dr. Frami re...","Dear [FIRSTNAME_1], Your appointment with Dr. ...",(842)-6372460


In [62]:
phone_numbers = df['phone_number'].tolist()
phone_numbers

target = []

for text in phone_numbers:
    if is_phone_number(text):
        target.append('TEL')
    else:
        target.append('O')

count_O = target.count('O')
count_TEL = target.count('TEL')

print(f'Number of "O": {count_O}')
print(f'Number of "TEL": {count_TEL}')

Number of "O": 1836
Number of "TEL": 581


In [63]:
df['target'] = target
df.head(10)

,unmasked_text,masked_text,phone_number,target
0,"Mr. Franecki, we have scheduled a learning ass...","[PREFIX_1] [LASTNAME_1], we have scheduled a l...",(612).3785804,O
1,The palliative care team is coordinating a hom...,The palliative care team is coordinating a hom...,09.48-67 61 36,O
2,"We've updated your next appointment, Tavares. ...","We've updated your next appointment, [FIRSTNAM...",+19-302 725.8274,TEL
3,Lawrence County Sports department is conductin...,[COUNTY_1] Sports department is conducting a p...,+97-259 289.7050,TEL
4,"Dear Ivory, Your appointment with Dr. Frami re...","Dear [FIRSTNAME_1], Your appointment with Dr. ...",(842)-6372460,O
5,"Hi Gerardo, here's your confirmation for your ...","Hi [FIRSTNAME_1], here's your confirmation for...",(017)-0128665,O
6,For property laws concerning Charley Graham at...,For property laws concerning [FIRSTNAME_1] [LA...,1066-121.1337,TEL
7,"Dear Mr. Hildegard Marlowe Borer, \nI trust th...",Dear [PREFIX_1] [FIRSTNAME_1] [MIDDLENAME_1] [...,+717-584.257-0232,O
8,"Hello Kenny, A follow-up psycho-oncology asses...","Hello [FIRSTNAME_1], A follow-up psycho-oncolo...",04056 81940,TEL
9,"Ms.'s appointment at [61.857,-105.1437] on Mar...",[PREFIX_1]'s appointment at [NEARBYGPSCOORDINA...,023 4733393,TEL


In [ ]:
df[['phone_number', 'target']]

In [ ]:
# Import shrieyaa_mini_df for checking false positive rows
filepath = os.path.join(os.getcwd(), "data", "shrieyaa_mini_df.csv")

# Read the CSV file using pandas
df = pd.read_csv(filepath)

In [64]:
df.to_csv('extracted_tel_numbers_english_updated.csv')

In [58]:
# 10/3 previous code

# Regex pattern to extract potential phone numbers from text
phone_pattern = re.compile(r'[\d\s\-\.\(\)]{7,}')
# Function to extract and validate phone numbers
def extract_validate_phone(text):
    potential_phone = phone_pattern.findall(text)
    valid_phone = [num.strip() for num in potential_phone if is_phone_number(num.strip())]
    return valid_phone
# Apply the function to each row in the df
df['valid_phone'] = df['unmasked_text'].apply(extract_validate_phone)

# Display rows where valid phone numbers were found
valid_phone_df = df[df['valid_phone'].apply(len) > 0]

valid_phone_df

# Convert valid_phone_df into a csv file
valid_phone_df.to_csv('valid_phone_numbers.csv')